In [ ]:
import openai
import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from extract_text import extract

%config InlineBackend.figure_formats = {'png', 'retina'}

# 正規化をする関数

In [ ]:
def normalization1(x):
    '''0から1の範囲に正規化をする関数'''
    x_min = np.min(x, axis=(0, 1))
    x_max = np.max(x, axis=(0, 1))
    normalized_x = (x - x_min) / (x_max- x_min)
    
    return normalized_x

In [ ]:
def normalization255(x):
    '''0から255の範囲に正規化をする関数'''
    x_min = np.min(x, axis=(0, 1))
    x_max = np.max(x, axis=(0, 1))    
    normalized_x = 255 * (x - x_min) / (x_max- x_min)
    
    return normalized_x.astype('uint8')

# 青空文庫から文章を取得

In [85]:
folder_name = 'sisei'   # フォルダ名を指定
url = 'https://www.aozora.gr.jp/cards/001383/files/56641_ruby_59456.zip'  # 青空文庫のURLを指定

folder_path = os.path.join('embeddings', folder_name)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)   # フォルダを作成

texts = extract(url).split('。')    # 青空文庫から文章を取得し、読点で分割
print(f'Total iteration: {len(texts)}')

Total iteration: 127


# 文章の埋め込みベクトル化

In [84]:
# OpenAIのAPI Keyをテキストファイルから取得
try:
    openai.api_key = os.environ['OPENAI_API_KEY']
except:
    with open('api_key.txt', 'r') as f:
        openai.api_key = f.read()
    
embeddings = []

# APIの呼び出し
for index, text in enumerate(texts):
    print(f'Iteration {index + 1}/{len(texts)} ', end='')
    
    response = openai.Embedding.create(
        input=text,
        model='text-embedding-ada-002'
    )
    
    embeddings.append(np.array(response['data'][0]['embedding'])) # APIレスポンスから埋め込みを取得
    print('completed')
    
    if index % 60 == 0:
        print('waiting for 60 seconds...')
        time.sleep(60)

embeddings = np.vstack(embeddings)

# 埋め込みベクトルをファイルへ保存
with open(os.path.join(folder_path, 'embeddings.pickle'), 'wb') as f:
    pickle.dump(embeddings, f)

Iteration 1/127 

APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 08 Nov 2023 13:47:03 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '822e43d84ed88316-KIX', 'alt-svc': 'h3=":443"; ma=86400'}

# 埋め込みベクトルを読み込み

In [ ]:
with open(f'{folder_name}/embeddings.pickle', 'rb') as f:
    embeddings = pickle.load(f)

# モノクロ画像を生成

In [ ]:
normalized_embeddings = normalization255(embeddings)    # ベクトルを正規化
image = Image.fromarray(normalized_embeddings)   # 行列を画像に変換
image.save(f'{folder_name}/embeddings_monochrome.png')
image

# T-SNEによる次元削減

## ベクトルから画像を生成

In [ ]:
tsne_embeddings = TSNE(n_components=2).fit_transform(embeddings)    # T-SNEで次元削減

In [ ]:
x = tsne_embeddings[:, 0]
y = tsne_embeddings[:, 1]

fig, ax = plt.subplots()
ax.plot(x, y, 'o')

In [ ]:
tsne_embeddings = TSNE(n_components=3).fit_transform(embeddings)    # T-SNEで次元削減
normalized_tsne_embeddings = normalization255(tsne_embeddings)   # 次元削減したベクトルを正規化
image_matrix = normalized_tsne_embeddings.reshape((len(normalized_tsne_embeddings), 1, 3))
image_matrix = np.concatenate([image_matrix for _ in range(2000)], 1)
image = Image.fromarray(image_matrix)

# 画像を保存して出力
image.save(f'{folder_name}/embeddings_band_tsne.png')
image

## ベクトルを3次元空間に射影

In [ ]:
normalized_tsne_embeddings = normalization1(tsne_embeddings)    # 次元削減したベクトルを正規化

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
ax.scatter(normalized_tsne_embeddings[:, 0], normalized_tsne_embeddings[:, 1], normalized_tsne_embeddings[:, 2], c=np.column_stack((normalized_tsne_embeddings[:, 0], normalized_tsne_embeddings[:, 1], normalized_tsne_embeddings[:, 2])))

def init():
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(normalized_tsne_embeddings[:, 0], normalized_tsne_embeddings[:, 1], normalized_tsne_embeddings[:, 2], c=np.column_stack((normalized_tsne_embeddings[:, 0], normalized_tsne_embeddings[:, 1], normalized_tsne_embeddings[:, 2])))
    return fig,

def animate(i):
    ax.view_init(elev=30., azim=3.6*i)
    return fig,

# アニメーションを作成して保存
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=100, interval=100, blit=True)
ani.save(f'{folder_name}/rotate_tsne.gif', writer='pillow', dpi=200)

# PCAによる次元削減

## ベクトルから画像を生成

In [ ]:
pca_embeddings = PCA(n_components=3).fit_transform(embeddings)  # PCAで次元削減
normalized_pca_embeddings = normalization255(pca_embeddings)   # 次元削減したベクトルを正規化
image_matrix = normalized_pca_embeddings.reshape((len(normalized_pca_embeddings), 1, 3))
image_matrix = np.concatenate([image_matrix for _ in range(2000)], 1)
image = Image.fromarray(image_matrix)

# 画像を保存して出力
image.save(f'{folder_name}/embeddings_band_pca.png')
image

## ベクトルを3次元空間に射影

In [ ]:
normalized_pca_embeddings = normalization1(pca_embeddings)  # 次元削減したベクトルを正規化

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
ax.scatter(normalized_pca_embeddings[:, 0], normalized_pca_embeddings[:, 1], normalized_pca_embeddings[:, 2], c=np.column_stack((normalized_pca_embeddings[:, 0], normalized_pca_embeddings[:, 1], normalized_pca_embeddings[:, 2])))

def init():
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(normalized_pca_embeddings[:, 0], normalized_pca_embeddings[:, 1], normalized_pca_embeddings[:, 2], c=np.column_stack((normalized_pca_embeddings[:, 0], normalized_pca_embeddings[:, 1], normalized_pca_embeddings[:, 2])))
    return fig,

def animate(i):
    ax.view_init(elev=30., azim=3.6*i)
    return fig,

# アニメーションを作成して保存
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=100, interval=100, blit=True)
ani.save(f'{folder_name}/rotate_pca.gif', writer='pillow', dpi=200)